In [1]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
import boto3
import sys
import re
from datetime import datetime
import configparser

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1662541423763_0006,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
%%configure -f
{
    "conf": {
        "spark.jars": "s3://sjet-datamart-bucket/Jars/mssql-jdbc-6.1.0.jre8.jar"        
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1662614190151_0006,pyspark,busy,Link,Link,


In [2]:
ARMSDataDF = spark.read.csv('s3://sjet-datamart-bucket/precog/pilot-info/', header=True)

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1662614190151_0020,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
dftest = spark.read.parquet('s3://sjet-datamart-bucket/precog/precog-processed-data1/q400/q400128wps/year=2021/month=December/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
dftest.show(5,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----------------+----------------+----------+--------+------+---------+------+---------------+-----+-------------------+-------------------+-------------------+------+------------+-------------+-------+------+---------+-------+--------+----------+------------+------------+---------------------------+--------------+----------------+--------------+-------------+--------------+--------+--------+-------------------+--------------------+-------------------------+---------------------+-------------------+----------------------+------------+----------------------+-------------+-----------------+-----------------+---------------+---------------+-----------------+-----------------+-----------------+-----------------+------------------+---------------------+----------------------+---------------+---------------+--------------------------+----------------------------+--------------+--------------+-----------------+-----------------+----------+----------+----------------+---------------

In [4]:
ARMSDataDF.createOrReplaceTempView('ARMSDataDF')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
df = spark.sql('''SELECT * FROM precogdb.boeing_3_256_wps WHERE year = '2021' and month = 'November' ''')
df.createOrReplaceTempView('df')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
#df.show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
#df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
Arms_Join_DF = spark.sql('''
                                        SELECT
                                            
                                            DF.*,
                                            CAST(AD.ChocksOff AS timestamp) AS ChocksOff,
                                            CAST(AD.ChocksOn AS timestamp) AS ChocksOn
                                           
                                                
                                        FROM df DF
                                        INNER JOIN ARMSDataDF AD ON DF.FlightNo=AD.DesigFlightNumber
                                                            AND DF.Date=AD.FlightDate
                                                            AND DF.Departure=AD.FromCity
                                                            AND DF.Arrival=AD.Tocity
                                    ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
#Arms_Join_DF.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
Arms_Join_DF.createOrReplaceTempView('Test')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
#spark.sql('''SELECT DATEADD(ss, 4000, 'ChockOff') AS DateAdd 
                #FROM Test ''').show(1, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
from pyspark.sql.functions import *
from pyspark.sql.types import StringType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
df2 = Arms_Join_DF.withColumn("timestamp_1", unix_timestamp(col("ChocksOff")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
df2.createOrReplaceTempView('test2')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
df3 = spark.sql('''
                    SELECT * , CAST(`time_secs` AS INT) + CAST(timestamp_1 AS INT) AS Time1 
                    FROM test2 
                ''')
#df3.show(1,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [77]:
df3.createOrReplaceTempView('test3')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [78]:
df4 = df3.withColumn("CurrentTime", col("Time1").cast("timestamp"))
df4.createOrReplaceTempView('test3')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
#df4.select('te_flap_posn-right').distinct().show(5,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+
|te_flap_posn-right|
+------------------+
|1.0               |
|14.2              |
|29.4              |
|28.7              |
|14.4              |
+------------------+
only showing top 5 rows

In [79]:
df4.write.mode("append").partitionBy("year", "month").parquet('s3://sjet-datamart-bucket/precog/final/boeing/boeing-3-256-wps/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
df2.show(5,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----------------+----------------+----------+--------+------+---------+------+----------------+-----+------+------------+-------------+-------+------------+------+---------+-------+--------+----------+------------+------------+---------------------------+--------------+----------------+--------------+-------------+--------------+--------+--------+-------------------+--------------------+-------------------------+---------------------+-------------------+----------------------+------------+----------------------+-------------+-----------------+-----------------+---------------+---------------+-----------------+-----------------+-----------------+-----------------+------------------+---------------------+----------------------+---------------+---------------+--------------------------+----------------------------+--------------+--------------+-----------------+-----------------+----------+----------+----------------+-----------------+-------------------------+-----------------